In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error


In [2]:
training_file = './input/train.csv'
training_data = pd.read_csv(training_file)

In [3]:
training_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
y = training_data.Survived

In [5]:
y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [6]:
# I'm going to put my data adjustments here.  I'll need to use this on both the training and test data.
def adjust_data(td):
    # I'm using a convention that derived features start with lowercase letter.
    td['male'] = (td['Sex'] == 'male') * 1.0
    td['female'] = (td['Sex'] == 'female') * 1.0
    td['cherbourg'] = (td['Embarked'] == 'C') * 1.0
    td['queenstown'] = (td['Embarked'] == 'Q') * 1.0
    td['southampton'] = (td['Embarked'] == 'S') * 1.0
    

In [7]:
adjust_data(training_data)
training_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,male,female,cherbourg,queenstown,southampton
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1.0,0.0,0.0,0.0,1.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0.0,1.0,1.0,0.0,0.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0.0,1.0,0.0,0.0,1.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0.0,1.0,0.0,0.0,1.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1.0,0.0,0.0,0.0,1.0


In [8]:
training_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
male             0
female           0
cherbourg        0
queenstown       0
southampton      0
dtype: int64

In [9]:
# declare features to be used in model
features = ['PassengerId', 'Pclass', 'SibSp', 'Parch', 'male', 
            'female', 'cherbourg', 'queenstown', 'southampton']
# Age Fare excluded for now due to NaN
X = training_data[features]
print('training size', training_data.shape)
print('filtered size', X.shape)
X.head()

training size (891, 17)
filtered size (891, 9)


,PassengerId,Pclass,SibSp,Parch,male,female,cherbourg,queenstown,southampton
0,1,3,1,0,1.0,0.0,0.0,0.0,1.0
1,2,1,1,0,0.0,1.0,1.0,0.0,0.0
2,3,3,0,0,0.0,1.0,0.0,0.0,1.0
3,4,1,1,0,0.0,1.0,0.0,0.0,1.0
4,5,3,0,0,1.0,0.0,0.0,0.0,1.0


In [10]:
X.isnull().sum()

PassengerId    0
Pclass         0
SibSp          0
Parch          0
male           0
female         0
cherbourg      0
queenstown     0
southampton    0
dtype: int64

In [11]:
# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

In [12]:
model = LogisticRegression(tol=1e-5, random_state=1, solver='liblinear', max_iter=200, verbose=1)

In [13]:
model.fit(train_X, train_y)

[LibLinear]

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=200, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=1, solver='liblinear',
          tol=1e-05, verbose=1, warm_start=False)

In [14]:
predictions = model.predict(val_X)

In [15]:
error = mean_squared_error(predictions, val_y)
print('Error is ', error)

Error is  0.22869955156950672


In [22]:
# For first submission without age or fare, error is 0.22869955156950672

In [16]:
correct = (val_y == predictions).count()
print('Correct predictions ', correct, ' out of ', predictions.size)

Correct predictions  223  out of  223


In [17]:
# now do the test predictions
test_file = './input/test.csv'
test_data = pd.read_csv(test_file)
adjust_data(test_data)
test_X = test_data[features]
test_X.head()

,PassengerId,Pclass,SibSp,Parch,male,female,cherbourg,queenstown,southampton
0,892,3,0,0,1.0,0.0,0.0,1.0,0.0
1,893,3,1,0,0.0,1.0,0.0,0.0,1.0
2,894,2,0,0,1.0,0.0,0.0,1.0,0.0
3,895,3,0,0,1.0,0.0,0.0,0.0,1.0
4,896,3,1,1,0.0,1.0,0.0,0.0,1.0


In [18]:
test_X.isnull().sum()

PassengerId    0
Pclass         0
SibSp          0
Parch          0
male           0
female         0
cherbourg      0
queenstown     0
southampton    0
dtype: int64

In [20]:
output = pd.DataFrame({'PassengerId': test_X.PassengerId,
                       'Survived': test_y})
output.to_csv('submission.csv', index=False)

In [21]:
output.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
